In [3]:
import itertools
import random
import collections

In [18]:
def readGraph(path):
    problem_read = False
    n = 0
    m = 0
    result = None
    with open(path, 'r') as fin:
        for line in fin:
            if line[0] == 'c':
                continue
            if line[0] == 'p':
                assert not problem_read
                problem_read = True
                assert line[0:5] == 'p cep'
                parts = line.split(' ')
                assert len(parts) == 4
                n = int(parts[2])
                m = int(parts[3])
                result = {u : set() for u in range(n)}
            else:
                assert problem_read
                parts = line.split(' ')
                assert(len(parts) == 2)
                u, v = map(int, parts)
                u -= 1
                v -= 1
                result[u].add(v)
                result[v].add(u)
    assert sum(map(len, result.values())) == 2 * m
    return result

In [25]:
def subGraph(g, nodes):
    return {u : {v for v in g[u] if v in nodes} for u in nodes}

In [35]:
def degeneracyOrdering(g):
    result = []
    degree = {u : len(g[u]) for u in g}
    while len(degree) > 0:
        # TODO: this is quadratic, replace by PQ!
        u = min(degree, key=lambda x : degree[x])
        for v in g[u]:
            if v in degree:
                degree[v] -= 1
        del degree[u]
        result.append(u)
    return result

In [40]:
def coloring(g):
    color = dict()
    colored_nodes = collections.defaultdict(list)
    order = degeneracyOrdering(g)
    order.reverse()
    for u in order:
        neighbor_colors = {color[v] for v in g[u] if v in color}
        c = 0
        while c in neighbor_colors:
            c += 1
        color[u] = c
        colored_nodes[c].append(u)
    return colored_nodes

In [22]:
def independentSet(g):
    result = set()
    # Greedy degree heuristic
    for u in sorted(g.keys(), key=lambda x : len(g[x])):
        if not any(v in result for v in g[u]):
            result.add(u)
    # 10 rounds two-improvements and prefer lower-degree replacements when possible
    for i in range(10):
        result_list = list(result)
        random.shuffle(result_list)
        for u in result_list:
            result.remove(u)
            candidate_found = False
            single_candidates = []
            for v in g[u]:
                if not any((x in result for x in g[v])):
                    result.add(v)
                    single_candidates.append(v)
                    for w in g[u]:
                        if w not in result and not any((x in result for x in g[w])):
                            result.add(w)
                            candidate_found = True
                    if candidate_found:
                        break
                    else:
                        result.remove(v)
            if not candidate_found:
                if single_candidates:
                    result.add(min(single_candidates, key=lambda x: len(g[x])))
                else:
                    result.add(u)
    # Ensure that we actually calculated an independent set
    for u in result:
        assert(not any((v in result for v in g[u])))
    return result

In [23]:
independentSet(graph)

{3, 12, 17, 22, 25, 38, 40, 47}

In [ ]:
def lowerBound(g):
    # All node pairs that are already covered by the bound
    bound_pairs = set()
    bound = 0
    stars = []
    # Start with highest-degree node
    for i, u in enumerate(sorted(g.keys(), reverse=False, key=lambda x: len(g[x]))):
        # Exclude nodes where (u, v) is in the bound
        candidates = [v for v in g[u] if (u, v) not in bound_pairs]
        neighborgraph = subGraph(g, candidates)
        # Add edges for all node pairs that are in the bound
        for x, y in itertools.combinations(candidates, 2):
            if (x, y) in bound_pairs:
                neighborgraph[x].add(y)
                neighborgraph[y].add(x)
        while True:
            # A star where all node pairs are not covered can be added to the bound
            star = independentSet(neighborgraph)
            if len(star) < 2:
                break
            bound += len(star) - 1
            for x in star:
                for y in neighborgraph[x]:
                    neighborgraph[y].remove(x)
                del neighborgraph[x]
            star = [u] + list(star)
            stars.append(star)
            for x, y in itertools.combinations(star, 2):
                bound_pairs.add((x, y))
                bound_pairs.add((y, x))
        #print(i, u, bound)
    #print(stars)
    return bound

In [69]:
def lowerBoundColoring(g):
    # All node pairs that are already covered by the bound
    bound_pairs = set()
    bound = 0
    stars = [[] for u in g]
    # Start with highest-degree node
    for i, u in enumerate(sorted(g.keys(), reverse=True, key=lambda x: len(g[x]))):
        # Exclude nodes where (u, v) is in the bound
        candidates = [v for v in g[u] if (u, v) not in bound_pairs]
        neighborgraph = subGraph(g, candidates)
        # Add edges for all node pairs that are in the bound
        for x, y in itertools.combinations(candidates, 2):
            if (x, y) in bound_pairs:
                neighborgraph[x].add(y)
                neighborgraph[y].add(x)
        # A star where all node pairs are not covered can be added to the bound
        u_stars = coloring(neighborgraph)
        for star in u_stars.values():
            if len(star) < 2:
                continue
            bound += len(star) - 1
            star = [u] + list(star)
            stars[u].append(star)
            for x, y in itertools.combinations(star, 2):
                bound_pairs.add((x, y))
                bound_pairs.add((y, x))
        #print(i, u, bound)
    random_nodes = list(g.keys())
    for i in range(10):
        random.shuffle(random_nodes)
        for u in random_nodes:
            for star in stars[u]:
                if len(star) > 2:
                    nodes_removed = set()
                    for v in star[1:]:
                        assert u != v
                        # Try removing v from star and inserting at least two P3
                        for x in set(star) - nodes_removed:
                            if x != v:
                                bound_pairs.remove((v, x))
                                bound_pairs.remove((x, v))
                        candidates_per_pair = dict()
                        for x in set(star) - nodes_removed:
                            if x != v:
                                if v in g[x]:
                                    assert(x == u)
                                    candidates_per_pair[(v, x)] = \
                                        [[v, x, y] for y in g[v] if y != x and y not in g[x] and (v, y) not in bound_pairs and (x, y) not in bound_pairs] + \
                                        [[x, v, y] for y in g[x] if y != v and y not in g[v] and (v, y) not in bound_pairs and (x, y) not in bound_pairs]
                                else:
                                    candidates_per_pair[(v, x)] = [[y, v, x] for y in g[x] & g[v] if (y, x) not in bound_pairs and (y, v) not in bound_pairs]

                        two_found = False
                        for pair, candidates in candidates_per_pair.items():
                            assert not two_found
                            for candidate in candidates:
                                assert not two_found
                                for x, y in itertools.combinations(candidate, 2):
                                    bound_pairs.add((x, y))
                                    bound_pairs.add((y, x))

                                for pair2, candidates2 in candidates_per_pair.items():
                                    if pair == pair2:
                                        continue
                                    for candidate2 in candidates2:
                                        if not any(p in bound_pairs for p in itertools.combinations(candidate2, 2)):
                                            two_found = True
                                            for x, y in itertools.combinations(candidate2, 2):
                                                bound_pairs.add((x, y))
                                                bound_pairs.add((y, x))
                                            stars[candidate2[0]].append(candidate2)
                                            bound += 1
                                if two_found:
                                    stars[candidate[0]].append(candidate)
                                    nodes_removed.add(v)
                                    break
                                else:
                                    for x, y in itertools.combinations(candidate, 2):
                                        bound_pairs.remove((x, y))
                                        bound_pairs.remove((y, x))
                            if two_found:
                                break
                        # Re-insert v
                        if not two_found:
                            for x in star:
                                if x != v:
                                    bound_pairs.add((v, x))
                                    bound_pairs.add((x, v))

                    for x in nodes_removed:
                        star.remove(x)

    return bound

In [75]:
graph = readGraph("/home/michael/graphs/PACE2021/heur/heur017.gr")
lowerBoundColoring(graph)

6720

In [70]:
for name in ['exact043.gr',
 'exact045.gr',
 'exact051.gr',
 'exact053.gr',
 'exact069.gr',
 'exact081.gr',
 'exact083.gr',
 'exact091.gr',
 'exact093.gr',
 'exact099.gr',
 'exact101.gr',
 'exact103.gr',
 'exact141.gr',
 'exact167.gr',
 'exact169.gr',
 'exact179.gr',
 'exact181.gr',
 'exact183.gr',
 'exact187.gr',
 'exact191.gr',
 'exact193.gr',
 'exact195.gr',
 'exact197.gr']:
    graph = readGraph("/home/michael/graphs/PACE2021/exact/{}".format(name))
    print(name, lowerBoundColoring(graph))

exact043.gr 904
exact045.gr 992
exact051.gr 1407
exact053.gr 1353
exact069.gr 1848
exact081.gr 1916
exact083.gr 2607
exact091.gr 2842
exact093.gr 3614
exact099.gr 610
exact101.gr 3810
exact103.gr 4106
exact141.gr 655
exact167.gr 8729
exact169.gr 8525
exact179.gr 578
exact181.gr 1392
exact183.gr 2515
exact187.gr 3624
exact191.gr 16785
exact193.gr 19413
exact195.gr 2285
exact197.gr 15013


In [47]:
for name in ['exact043.gr',
 'exact045.gr',
 'exact051.gr',
 'exact053.gr',
 'exact069.gr',
 'exact081.gr',
 'exact083.gr',
 'exact091.gr',
 'exact093.gr',
 'exact099.gr',
 'exact101.gr',
 'exact103.gr',
 'exact141.gr',
 'exact167.gr',
 'exact169.gr',
 'exact179.gr',
 'exact181.gr',
 'exact183.gr',
 'exact187.gr',
 'exact191.gr',
 'exact193.gr',
 'exact195.gr',
 'exact197.gr']:
    graph = reader.read("/home/michael/graphs/PACE2021/exact/{}".format(name))
    print(name, lowerBound(graph))

exact043.gr 855
exact045.gr 943
exact051.gr 1358
exact053.gr 1259
exact069.gr 1788
exact081.gr 1782
exact083.gr 2465
exact091.gr 2707
exact093.gr 3361
exact099.gr 594
exact101.gr 3572
exact103.gr 3793
exact141.gr 615
exact167.gr 8399
exact169.gr 7777
exact179.gr 571
exact181.gr 1348
exact183.gr 2391
exact187.gr 3436
exact191.gr 14932
exact193.gr 16391
exact195.gr 2132
exact197.gr 13952
